# Rheinmetall (RHM) Return Analysis

**Author:** Velina Boneva  
**Goal:** Demonstrate practical Python workflow used in markets analysis.


In [47]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf

In [48]:
# 1) Download 5 years of daily data
rhm = yf.download("RHM.DE", period="5y", interval="1d", auto_adjust=False)

# 2) If Yahoo returned MultiIndex columns (Price / Ticker), flatten them
if isinstance(rhm.columns, pd.MultiIndex):
    # keep the first level: Adj Close, Close, High, Low, Open, Volume
    rhm.columns = rhm.columns.get_level_values(0)

# 3) Make Date a normal column
rhm = rhm.reset_index()

# 4) Keep and order columns exactly as you want
rhm = rhm[['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']]

rhm.columns.name = None

# 5) Check result
rhm

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Volume,Adj Close
0,2020-12-11,78.540001,78.940002,76.800003,78.639999,159395,72.997787
1,2020-12-14,79.040001,81.080002,79.040001,80.059998,195493,74.315910
2,2020-12-15,80.400002,83.779999,80.320000,83.279999,264759,77.304878
3,2020-12-16,83.779999,85.860001,83.639999,84.760002,330455,78.678696
4,2020-12-17,85.000000,86.440002,84.000000,86.080002,235750,79.903984
...,...,...,...,...,...,...,...
1271,2025-12-05,1539.000000,1560.000000,1512.500000,1529.500000,181006,1529.500000
1272,2025-12-08,1529.500000,1529.500000,1529.500000,1529.500000,0,1529.500000
1273,2025-12-09,1630.000000,1661.500000,1610.500000,1641.500000,281680,1641.500000
1274,2025-12-10,1629.000000,1632.000000,1568.500000,1610.000000,227522,1610.000000


In [50]:
import numpy as np
import pandas as pd

# Ensure Date is datetime
rhm['Date'] = pd.to_datetime(rhm['Date'])

# --- mark zero-volume rows (if any) ---
mask_zero = rhm['Volume'] == 0
print("Number of zero-volume rows:", mask_zero.sum())
print(rhm[mask_zero][['Date', 'Volume']])

# Only do the rest if there ARE zeros
if mask_zero.any():
    # Replace 0 with NaN
    rhm['Volume'] = rhm['Volume'].replace(0, np.nan)

    # Set index to Date for time interpolation
    rhm = rhm.set_index('Date')

    # Interpolate
    rhm['Volume'] = rhm['Volume'].interpolate(method='time')

    # Back to integer + restore Date column
    rhm['Volume'] = rhm['Volume'].round().astype(int)
    rhm = rhm.reset_index()

Number of zero-volume rows: 0
Empty DataFrame
Columns: [Date, Volume]
Index: []


In [51]:
rhm

,Date,Open,High,Low,Close,Volume,Adj Close
0,2020-12-11,78.540001,78.940002,76.800003,78.639999,159395,72.997787
1,2020-12-14,79.040001,81.080002,79.040001,80.059998,195493,74.315910
2,2020-12-15,80.400002,83.779999,80.320000,83.279999,264759,77.304878
3,2020-12-16,83.779999,85.860001,83.639999,84.760002,330455,78.678696
4,2020-12-17,85.000000,86.440002,84.000000,86.080002,235750,79.903984
...,...,...,...,...,...,...,...
1271,2025-12-05,1539.000000,1560.000000,1512.500000,1529.500000,181006,1529.500000
1272,2025-12-08,1529.500000,1529.500000,1529.500000,1529.500000,256512,1529.500000
1273,2025-12-09,1630.000000,1661.500000,1610.500000,1641.500000,281680,1641.500000
1274,2025-12-10,1629.000000,1632.000000,1568.500000,1610.000000,227522,1610.000000


In [52]:
rhm['Daily Return'] = rhm['Adj Close'].pct_change(1) * 100
rhm

,Date,Open,High,Low,Close,Volume,Adj Close,Daily Return
0,2020-12-11,78.540001,78.940002,76.800003,78.639999,159395,72.997787,NaN
1,2020-12-14,79.040001,81.080002,79.040001,80.059998,195493,74.315910,1.805702
2,2020-12-15,80.400002,83.779999,80.320000,83.279999,264759,77.304878,4.021976
3,2020-12-16,83.779999,85.860001,83.639999,84.760002,330455,78.678696,1.777142
4,2020-12-17,85.000000,86.440002,84.000000,86.080002,235750,79.903984,1.557332
...,...,...,...,...,...,...,...,...
1271,2025-12-05,1539.000000,1560.000000,1512.500000,1529.500000,181006,1529.500000,-0.681818
1272,2025-12-08,1529.500000,1529.500000,1529.500000,1529.500000,256512,1529.500000,0.000000
1273,2025-12-09,1630.000000,1661.500000,1610.500000,1641.500000,281680,1641.500000,7.322654
1274,2025-12-10,1629.000000,1632.000000,1568.500000,1610.000000,227522,1610.000000,-1.918977


In [53]:
rhm['Daily Return'] = rhm['Daily Return'].replace(np.nan,0)
rhm

,Date,Open,High,Low,Close,Volume,Adj Close,Daily Return
0,2020-12-11,78.540001,78.940002,76.800003,78.639999,159395,72.997787,0.000000
1,2020-12-14,79.040001,81.080002,79.040001,80.059998,195493,74.315910,1.805702
2,2020-12-15,80.400002,83.779999,80.320000,83.279999,264759,77.304878,4.021976
3,2020-12-16,83.779999,85.860001,83.639999,84.760002,330455,78.678696,1.777142
4,2020-12-17,85.000000,86.440002,84.000000,86.080002,235750,79.903984,1.557332
...,...,...,...,...,...,...,...,...
1271,2025-12-05,1539.000000,1560.000000,1512.500000,1529.500000,181006,1529.500000,-0.681818
1272,2025-12-08,1529.500000,1529.500000,1529.500000,1529.500000,256512,1529.500000,0.000000
1273,2025-12-09,1630.000000,1661.500000,1610.500000,1641.500000,281680,1641.500000,7.322654
1274,2025-12-10,1629.000000,1632.000000,1568.500000,1610.000000,227522,1610.000000,-1.918977


In [54]:
rhm.to_csv("Stocks and Portfolio Analysis.csv", index=False)

In [55]:
rhm['Date'] = pd.to_datetime(rhm['Date'])
rhm = rhm.set_index('Date')

In [56]:
rhm

,Open,High,Low,Close,Volume,Adj Close,Daily Return
Date,,,,,,,
2020-12-11,78.540001,78.940002,76.800003,78.639999,159395,72.997787,0.000000
2020-12-14,79.040001,81.080002,79.040001,80.059998,195493,74.315910,1.805702
2020-12-15,80.400002,83.779999,80.320000,83.279999,264759,77.304878,4.021976
2020-12-16,83.779999,85.860001,83.639999,84.760002,330455,78.678696,1.777142
2020-12-17,85.000000,86.440002,84.000000,86.080002,235750,79.903984,1.557332
...,...,...,...,...,...,...,...
2025-12-05,1539.000000,1560.000000,1512.500000,1529.500000,181006,1529.500000,-0.681818
2025-12-08,1529.500000,1529.500000,1529.500000,1529.500000,256512,1529.500000,0.000000
2025-12-09,1630.000000,1661.500000,1610.500000,1641.500000,281680,1641.500000,7.322654


In [57]:
rhm.describe().round(2)

,Open,High,Low,Close,Volume,Adj Close,Daily Return
count,1276.00,1276.00,1276.00,1276.00,1276.00,1276.00,1276.00
mean,486.49,493.53,477.35,486.29,246957.42,481.69,0.27
std,541.51,548.95,530.07,540.46,216474.09,541.92,2.55
min,77.00,77.40,76.28,77.00,35529.00,73.00,-13.79
25%,148.08,154.32,145.04,149.57,133857.00,143.81,-0.91
50%,253.20,256.10,250.05,253.20,190679.00,248.54,0.11
75%,526.40,534.40,517.85,527.45,282983.00,521.71,1.41
max,1993.50,2008.00,1988.50,1988.50,3484652.00,1988.50,24.80
